# Description: Predicting Ethereum Price with Python and Machine Learning

In [10]:
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta

import seaborn as sns
import numpy as np
import re 
# RegEx: Regular expression

import mplfinance as mpf
import pandas_datareader as web

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')






In [11]:
import pandas as pd
df = pd.read_csv('ethereum_2015-08-07_2024-06-02.csv')
df = df.set_index(pd.DatetimeIndex(df['Start'].values))
display(df)





,Start,End,Open,High,Low,Close,Volume,Market Cap
2024-06-01,2024-06-01,2024-06-02,2948.742856,2978.306538,2941.392797,2971.467374,1.519628e+10,3.558701e+11
2024-05-31,2024-05-31,2024-06-01,2940.157084,2992.084249,2925.939443,2947.755919,1.673715e+10,3.544224e+11
2024-05-30,2024-05-30,2024-05-31,2953.729354,2981.526565,2913.709414,2940.947921,1.754862e+10,3.542934e+11
2024-05-29,2024-05-29,2024-05-30,3000.900270,3021.146455,2938.025184,2951.882733,1.889175e+10,3.575360e+11
2024-05-28,2024-05-28,2024-05-29,3028.869403,3045.442442,2962.778842,3000.077482,1.966750e+10,3.614046e+11
...,...,...,...,...,...,...,...,...
2015-08-12,2015-08-12,2015-08-13,0.838300,1.012639,0.693657,0.971606,1.547950e+06,5.246035e+07
2015-08-11,2015-08-11,2015-08-12,0.556151,0.888188,0.520658,0.830301,5.029142e+05,3.743174e+07
2015-08-10,2015-08-10,2015-08-11,0.551009,0.572956,0.499706,0.555567,3.593449e+05,3.283582e+07
2015-08-09,2015-08-09,2015-08-10,0.591381,0.690675,0.493933,0.572558,5.493833e+05,3.262780e+07


In [12]:
future_days = 60

#Create a new column (target) shifted 'x' units/days up
df[str(future_days) + '_Day_Price_Forecast'] = df [['Close']].shift(-future_days)
#Show the data
display(df[['Close', str(future_days)+'_Day_Price_Forecast']])

,Close,60_Day_Price_Forecast
2024-06-01,2971.467374,2574.545783
2024-05-31,2947.755919,2756.781150
2024-05-30,2940.947921,2868.217516
2024-05-29,2951.882733,2762.624741
2024-05-28,3000.077482,2763.693242
...,...,...
2015-08-12,0.971606,NaN
2015-08-11,0.830301,NaN
2015-08-10,0.555567,NaN
2015-08-09,0.572558,NaN


In [22]:

x = np.array(df[['Close']])
x = x[:df.shape[0]-future_days]
print(x)


[[2.97146737e+03]
 [2.94775592e+03]
 [2.94094792e+03]
 ...
 [5.11384475e-01]
 [4.86041418e-01]
 [4.78269641e-01]]


In [24]:
y = np.array(df[[str(future_days)+'_Day_Price_Forecast']])
y = y[:-future_days]
print(y)


[[2.57454578e+03]
 [2.75678115e+03]
 [2.86821752e+03]
 ...
 [5.55566633e-01]
 [5.72557778e-01]
 [5.61197639e-01]]


In [26]:
#split the data into training and testing
from sklearn.model_selection import train_test_split    
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)